### Model:
The NN used in the paper is InceptionV1, known as GoogLeNet too. Its implementation is present and there is no need to re-implement it from scratch. In this document, we will be loading a version of it that is shared on Internet Via the pretrained models of Pytorch.<br>
The version that we will load is trained to classify images of ImageNet dataset.

In [115]:
#Loading of the libraries needed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import os
from tqdm import tqdm
import time
import numpy as np, random
from PIL import Image
import os
import torch.utils.data as data
from skimage import io, transform, img_as_float
import PIL
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 5
LEARNING_RATE = 1e-3
Image_size = 256 # This one is to be changed with the image size that we want to train with: (Image_size X Image_size)

In [116]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'googlenet', pretrained=True)
in_features = model.fc.in_features
# In the classification task, we have only 2 classes. 0 for "no Car" and 1 when "at least a car".
out_features = 2
model.fc = nn.Linear(in_features,out_features)
model.to(device)
print("Anything to omit showing architecture! ;p")

Using cache found in C:\Users\Utilisateur/.cache\torch\hub\pytorch_vision_v0.6.0


Anything to omit showing architecture! ;p


### Preparing necessary functions

In [133]:
#Definition of train and test functions!
def train(model, trainloader,epochs=2, lr=LEARNING_RATE):
    total_loss = 0
    # We need to verify whether we want to modify the Loss function. I believe we do but I am not sure yet.
    loss_function=nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        total_correct=0
        for batch_idx, (images, labels) in enumerate(trainloader):
            images=images.to(device)
            labels=labels.to(device)
            optimizer.zero_grad() 
            preds= model(images)
            loss = loss_function(preds,labels)
            loss.backward() 
            optimizer.step() 
            total_loss += loss.item()
    return model
def test(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [134]:
# Definition of DataSet creator, This needs to be revisited because, I do not have the format of the .txt file
class Dataset(torch.utils.data.Dataset):

    def __init__(self, number_images, root_dir, folder, label_file , transform=None):
        # Number of images to be loaded.
        self.number_images = number_images
        with open(root_dir+label_file, 'r') as file:
            lines=file.read().splitlines()
        self.Names_labels = []
        for element in lines:
            self.Names_labels.append(element.split(' '))
        
        # Here I construct the path to the images
        self.root_dir = root_dir+folder
        # this is to collect the transform, if I supply any.
        self.transform = transform

    def __len__(self):
        return self.number_images

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,self.Names_labels[idx][0].split('/')[-1])
        image = io.imread(img_name)
        resized_img = transform.resize(image, (Image_size, Image_size))
        sample = img_as_float(resized_img)
        target = int(float(self.Names_labels[idx][1]))

        if self.transform:
            sample = self.transform(sample)
        
        return [sample.float(), target]

### Loading Dataset
In this phase, we will only consider using the portion of dataset captured from Colombus city. The whole dataset will be used when everything is set correctly.

In [135]:
Path_train ="."
number_images_training = 6966 
transformation = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = Dataset(number_images=number_images_training, root_dir=Path_train, folder = '/train',
                        label_file = '/COWC_train_list_detection.txt', transform = transformation)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
Path_test = "."
number_images_testing = 2050
test_dataset = Dataset(number_images=number_images_testing, root_dir=Path_test, folder= '/test',
                       label_file = '/COWC_test_list_detection.txt', transform = transformation)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Training for Classification

In [137]:
# Normally if all the points above that needs to be filled are done, we can start the training without any further processing.
model = train(model, train_dataloader,epochs=5, lr=LEARNING_RATE)


100%|██████████| 5/5 [23:07<00:00, 277.57s/it]


In [138]:
test(model, test_dataloader)

75.26829268292683

### Training for Detection

### Training for Counting